In [ ]:
import pandas as pd
import dateutil.parser as dp
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import os
import glob
from collections import OrderedDict
import numpy as np

In [ ]:
frameworks = ['fuzzware_hal', 'fuzzware_nohal', 'halucinator', 'libafl_halucinator', 'safire']

palette = {"fuzzware_hal": "C0", 'fuzzware_nohal': 'C4', "halucinator": "C1", "libafl_halucinator": "C2", "safire": "C3", "dark": "black"}

framework_to_legend = {"fuzzware_hal": "Fuzzware", 'fuzzware_nohal': 'Fuzzware-NoHAL', "halucinator": "HALucinator", "libafl_halucinator": "HALucinator-libafl", "safire": "SAFIREFuzz"}


In [ ]:
targets = ['atmel_6lowpan_udp_rx', 'atmel_6lowpan_udp_tx', 'nxp_lwip_http', 'samr21_http', 'p2im_plc',  'p2im_drone', 'st-plc', 'wycinwyc', 'stm32_tcp_echo_client', 'stm32_tcp_echo_server', 'stm32_udp_echo_client', 'stm32_udp_echo_server']

target_to_label = ["6LoWPAN Receiver", "6LoWPAN Transmitter", "NXP HTTP", "SAMR21 HTTP", "P2IM PLC", "P2IM Drone", "STM PLC", "WYCINWYC", "TCP Echo Client", "TCP Echo Server", "UDP Echo Client", "UDP Echo Server"]

In [ ]:
def remove_duplicates(data):
    data_in = OrderedDict(data)
    data_out = {}
    for k, v in data_in.items():
        if v not in data_out.values():
            data_out[k] = v
    return OrderedDict(data_out)

def parse_fuzzware_data(data):
    data = data.split('\n')[1:]
    data = [(d.split('\t')[:2]) for d in data if d != '']
    data = {int(d[0]): int(d[1]) for d in data}
    return remove_duplicates(data)

def parse_safire_data(data):
    data = OrderedDict(eval(data))
    t0 = next(iter(data.keys()))
    t0 = int(dp.parse(t0).timestamp())
    data = {int(dp.parse(k).timestamp())-t0:v for k,v in data.items()}
    return remove_duplicates(data)
    

def parse_halucinator_data(data):
    data = eval(data)
    data = {int(k):v for k,v in data.items()}
    data = OrderedDict(data)
    t0 = next(iter(data.keys()))
    data = {k-t0:v for k,v in data.items()}
    return remove_duplicates(data)

# readall data
all_data = {}
for t in targets:
    all_data[t] = {}
    for fw in frameworks:
        print(f"Processing {t} for {fw}")

        all_data[t][fw] = {}
        files = glob.glob(f'{fw}_data/{fw}_{t}*.data')
        assert(len(files) == 5)
        for i in range(5):
            with open(files[i], 'r') as f:
                data = f.read()
            if 'fuzzware' in fw:
                data = parse_fuzzware_data(data)
            elif 'safire' in fw or 'libafl_halucinator' in fw:
                data = parse_safire_data(data)
            elif 'halucinator' in fw:
                data = parse_halucinator_data(data)
            else:
                assert("NOT REACHABLE")
            all_data[t][fw][f'run-{i}'] = data

In [ ]:
# fill the missing idxs
plot_dfs = {}

for t in targets:
    plot_dfs[t] = {}
    for fw in frameworks:
        plot_dfs[t][fw] = {}
        keys = set()
        for _, run in all_data[t][fw].items():
            keys |= run.keys()
        
        
        for i in range(5):
            new_dict = {k: np.nan for k in keys}
            new_dict.update(all_data[t][fw][f'run-{i}'])
            new_dict[86399] = np.nan # add end data point
            new_dict = OrderedDict(sorted(new_dict.items()))
            df = pd.DataFrame(new_dict.items(), columns=['time','coverage'])
            df['run'] = [f'run-{i}'] * len(df)
            df['fuzzer'] = [f'{fw}'] * len(df)

            df = df.ffill()
            plot_dfs[t][fw][f'run-{i}'] = df
    
        

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

i = 0
j = 0

#sns.set(font_scale=1.0, rc={'text.usetex' : True})
#sns.set_style("whitegrid")
sns.set_style("darkgrid")
#print(sns.color_palette("tab10").as_cmap())

for t, tname in zip(targets, target_to_label):
    print(f'Plotting {t}, {tname}')
    
    for fw in frameworks:
     

        
        df = pd.concat( [df for df in plot_dfs[t][fw].values()])
        #df.to_csv('/tmp/a.csv')
        sns.set_style("darkgrid")
        g = sns.lineplot(ax=axes[j,i],
            x='time',
            y="coverage",
            #style='fuzzer',
            hue='fuzzer',
            palette=palette,
            data=df,
            errorbar=('ci', 95),
            estimator=np.median,
            #dashes=False,
            )
                  
        g.set_title(tname)
        g.set(xlabel=None)
        g.set(ylabel=None)
        g.legend_.remove()
        g.set_xlim([1, 24*60*60+1])
        
        g.set(xscale="log")

        
    i = (i+1) % 4
    if i == 0:
        j += 1 


fig.text(0.54, 0.025, 'Time in Seconds', ha='center', fontsize=18)
fig.text(0.025, 0.55, '# Reached Basic Blocks', va='center', rotation='vertical', fontsize=18)
handles, labels = g.get_legend_handles_labels()
print(labels)

labels = [framework_to_legend[l] for l in labels]


plt.figlegend( handles, labels, ncol=6, labelspacing=0., bbox_to_anchor=(1, 1, -.22, .025) )
plt.tight_layout()
plt.subplots_adjust(bottom=0.09, left=0.09)
plt.savefig('coverage.png')